In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   --------------------------- ------------ 7.6/11.1 MB 36.2 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 38.7 MB/s eta 0:00:00


In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose #this is where we are importing the pre-built pose detection model from mediapipe


In [6]:
from IPython.display import HTML

# Define the HTML content with a corrected image URL
html_content = '<img src="https://editor.analyticsvidhya.com/uploads/77738pose_tracking_full_body_landmarks.png" style="height:300px">'

# Display the HTML content
display(HTML(html_content))


In [7]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [8]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

landmarks = ['class']
for val in range(1, 34): #because 33 points
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]


In [9]:
landmarks[1:]

['x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 'x33',
 'y3

In [33]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar= '"', quoting= csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)


In [34]:
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        keypoints = np.insert(keypoints, 0, action)  # Insert action at the beginning

        with open('coords.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print('Error occurred while exporting landmarks:', e)


In [49]:
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Video feed
cap = cv2.VideoCapture('training_data.mp4')  # Load video file

# Setup Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Stop if the video ends

        # Convert to RGB for Mediapipe processing
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False  # Save memory

        # Process image to detect pose
        results = pose.process(image)

        # Convert back to BGR for OpenCV display
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw pose landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display the video
        cv2.imshow('Pose Detection', image)

        # Capture keypress
        k = cv2.waitKey(1)
        if k == 117:  # Key 'u' for saving landmark type '1'
            export_landmark(results, '1')
        if k == 100:  # Key 'd' for saving landmark type '2'
            export_landmark(results, '2')

        # Exit on 'q' key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [51]:
df = pd.read_csv('coords.csv')

In [52]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,1.0,0.625842,0.382540,-0.278379,0.998219,0.626301,0.367036,-0.267018,0.998421,0.626316,...,0.112289,0.969579,0.556666,0.784352,0.625289,0.689475,0.557305,0.844543,0.089445,0.934783
1,1.0,0.663273,0.500832,-0.298777,0.999510,0.659862,0.487401,-0.282720,0.999593,0.658988,...,0.088629,0.956687,0.550015,0.786992,0.625923,0.616155,0.538087,0.860871,0.117298,0.962428
2,1.0,0.431886,0.370497,-0.129702,0.999678,0.413210,0.360921,-0.106775,0.999546,0.412733,...,0.020689,0.963287,0.481684,0.908725,0.482452,0.797194,0.442401,0.987611,-0.159918,0.954868
3,1.0,0.450770,0.374052,-0.113100,0.999725,0.434039,0.364901,-0.093964,0.999608,0.433274,...,0.055226,0.961285,0.475380,0.913568,0.446953,0.816030,0.439364,0.984781,-0.117380,0.953458
4,1.0,0.660806,0.499839,-0.335329,0.999325,0.657824,0.485978,-0.323548,0.999430,0.657278,...,0.138320,0.959791,0.551115,0.782549,0.714453,0.626996,0.537588,0.858446,0.150409,0.961965


In [53]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("coords.csv")

# Get the total number of rows
num_rows = len(df)

print(f"Total number of rows: {num_rows}")


Total number of rows: 65


In [27]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point (joint)
    c = np.array(c)  # Last point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Track reps and squat state
rep_count = 0
squat_state = "UP"  # Start in UP position

# Load Video
cap = cv2.VideoCapture('training_data.mp4')

# Start Pose Detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Stop if the video ends

        # Convert to RGB for Mediapipe processing
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False  # Save memory

        # Process image to detect pose
        results = pose.process(image)

        # Convert back to BGR for OpenCV display
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            h, w, _ = image.shape  # Get frame dimensions

            # Get coordinates of relevant joints
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * w,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * h]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x * w,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y * h]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x * w,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y * h]

            # ✅ Calculate knee angle for rep counting
            knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

            # ✅ Rep Counting Using Knee Angle
            if knee_angle > 160:  # Standing position (UP)
                squat_state = "UP"
            elif knee_angle < 90:  # Deep squat position (DOWN)
                if squat_state == "UP":  # If moving from UP → DOWN, count rep
                    rep_count += 1
                    print(f"✅ Rep Count: {rep_count}")
                squat_state = "DOWN"

            # ✅ Draw pose landmarks
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # ✅ Display knee angle at knee position
            knee_x, knee_y = int(left_knee[0]), int(left_knee[1])
            cv2.putText(image, f"{int(knee_angle)}°", (knee_x, knee_y), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)

            # ✅ Display Rep Counter in the Corner
            cv2.rectangle(image, (0, 0), (200, 60), (245, 117, 16), -1)
            cv2.putText(image, f"Reps: {rep_count}", (10, 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Display the video
        cv2.imshow('Squat Counter', image)

        # Exit on 'q' key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()


✅ Rep Count: 1
✅ Rep Count: 2
✅ Rep Count: 3
✅ Rep Count: 4
✅ Rep Count: 5
✅ Rep Count: 6
✅ Rep Count: 7
✅ Rep Count: 8
✅ Rep Count: 9
✅ Rep Count: 10
✅ Rep Count: 11
✅ Rep Count: 12
✅ Rep Count: 13
✅ Rep Count: 14
✅ Rep Count: 15
✅ Rep Count: 16
✅ Rep Count: 17
✅ Rep Count: 18
✅ Rep Count: 19
✅ Rep Count: 20
✅ Rep Count: 21
✅ Rep Count: 22
✅ Rep Count: 23
✅ Rep Count: 24
✅ Rep Count: 25


In [28]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point (joint)
    c = np.array(c)  # Last point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Track reps and squat state
rep_count = 0
squat_state = "UP"  # Start in UP position
feedback = ""  # Form feedback
feedback_good_bad = "good"  # "good" or "bad"

# Load Video
cap = cv2.VideoCapture('training_data.mp4')

# Start Pose Detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Stop if the video ends

        # Convert to RGB for Mediapipe processing
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False  # Save memory

        # Process image to detect pose
        results = pose.process(image)

        # Convert back to BGR for OpenCV display
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            h, w, _ = image.shape  # Get frame dimensions

            # Get coordinates of relevant joints
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x * w,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y * h]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * w,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * h]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x * w,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y * h]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x * w,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y * h]

            # ✅ Calculate angles for feedback
            angle_hip_knee = calculate_angle(left_wrist, left_hip, left_knee)
            angle_knee_ankle = calculate_angle(left_hip, left_knee, left_ankle)
            angle_diff = angle_hip_knee - angle_knee_ankle

            # ✅ Form Feedback Logic
            if angle_knee_ankle < 80:  # Deep squat position
                if angle_diff < -10:
                    feedback = "Keep Chest Up"
                    feedback_good_bad = "bad"
                else:
                    feedback = "Good Form"
                    feedback_good_bad = "good"
            else:
                feedback = ""

            # ✅ Rep Counting Using Knee Angle
            if angle_knee_ankle > 160:  # Standing position (UP)
                squat_state = "UP"
            elif angle_knee_ankle < 90:  # Deep squat position (DOWN)
                if squat_state == "UP":  # If moving from UP → DOWN, count rep
                    rep_count += 1
                    print(f"✅ Rep Count: {rep_count}")
                squat_state = "DOWN"

            # ✅ Draw pose landmarks
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # ✅ Display knee angle at knee position
            knee_x, knee_y = int(left_knee[0]), int(left_knee[1])
            cv2.putText(image, f"{int(angle_knee_ankle)}°", (knee_x, knee_y), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)

            # ✅ Display Form Feedback & Rep Counter
            cv2.rectangle(image, (0, 0), (300, 100), (245, 117, 16), -1)  # Background

            cv2.putText(image, f"Reps: {rep_count}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, f"Feedback: {feedback}", (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if feedback_good_bad == "good" else (0, 0, 255), 2, cv2.LINE_AA)

            cv2.putText(image, f"Form: {feedback_good_bad}", (10, 90), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if feedback_good_bad == "good" else (0, 0, 255), 2, cv2.LINE_AA)

        # Display the video
        cv2.imshow('Squat Counter & Form Feedback', image)

        # Exit on 'q' key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()


✅ Rep Count: 1
✅ Rep Count: 2
✅ Rep Count: 3
✅ Rep Count: 4
✅ Rep Count: 5
✅ Rep Count: 6
✅ Rep Count: 7
✅ Rep Count: 8
✅ Rep Count: 9
✅ Rep Count: 10


KeyboardInterrupt: 